In [3]:
import (
    "bufio"
    "fmt"
    "strings"
    "strconv"
    "os"
    "math"
    "io/ioutil"
    "encoding/json"
)

type Packet struct {
    Version int
    Type int
    LiteralValue int
    SubPackets []*Packet
}

var INITIAL_QUEUE_SIZE = 0
var VersionSum int = 0

func PrettyPrint(v interface{}) (err error) {
      b, err := json.MarshalIndent(v, "", "  ")
      if err == nil {
              fmt.Println(string(b))
      }
      return
}

func getBits(ch <-chan string, numBits int) uint64 {
    // reads numBits number of items from the channel
    // and returns an integer representation of the "bits"
    if len(ch) < numBits {
        return 0
    }
    var bStr string = ""
    for i := 0; i < numBits; i++ {
        bStr += <- ch
    }
    iVal,_ := strconv.ParseUint(bStr,2,32)
    return iVal
}

func loadData(file string) chan string{
    f, err := ioutil.ReadFile(file)
    rawData := strings.Split(string(f),"")
    ch := make(chan string, len(rawData)*4)
    for i,sVal := range rawData {
        intVal, _ := strconv.ParseUint(strings.TrimSuffix(sVal, "\n"), 16, 32)
        bStr := fmt.Sprintf("%04b",intVal)
        for _,v := range bStr {
            ch <- string(v)
        }
    }
    return ch
}

func getLiteral(ch <-chan string) int {
    tFlag := getBits(ch,1)
    valParts := []uint64{}
    for tFlag != 0 {
        valParts = append(valParts, getBits(ch,4))
        tFlag = getBits(ch,1)
    }
    valParts = append(valParts,getBits(ch,4))
    litVal := 0
    for i,v := range valParts {
        litVal += int(v << ((len(valParts)-1-i)*4))
    }
    return litVal
}

func getOperation(ch <-chan string) []*Packet {
    lenTypeID := getBits(ch,1)
    subPackets := []*Packet{}
    switch lenTypeID {
    case 0:
        tLen := int(getBits(ch,15))
        finished := len(ch) - tLen
        remaining := len(ch) - finished
        pVer := int(getBits(ch,3))
        pType := int(getBits(ch,3))
        for remaining != 0 {
            packet := new(Packet)
            packet.Version = pVer
            packet.Type = pType
            VersionSum += pVer
            if packet.Type == 4 {
                packet.LiteralValue = getLiteral(ch)
                remaining = len(ch) - finished
                subPackets = append(subPackets,packet)
            } else {
                packet.SubPackets = append(packet.SubPackets, getOperation(ch)...)
                remaining = len(ch) - finished
                subPackets = append(subPackets,packet)
            }
            remaining = len(ch) - finished
            if remaining > 6 {
                pVer = int(getBits(ch,3))
                pType = int(getBits(ch,3))
            } else {
                devNull := int(getBits(ch,remaining))
            }
            remaining = len(ch) - finished
        }
    case 1:
        stLen := len(ch)
        numSP := int(getBits(ch,11))
        var pVer int
        var pType int
        for i := 0 ; i < numSP; i++ {
            pVer = int(getBits(ch,3))
            pType = int(getBits(ch,3))
            packet := new(Packet)
            packet.Version = pVer
            packet.Type = pType
            VersionSum += pVer
            if packet.Type == 4 {
                packet.LiteralValue = getLiteral(ch)
                subPackets = append(subPackets,packet)
            } else {
                packet.SubPackets = append(packet.SubPackets, getOperation(ch)...)
                subPackets = append(subPackets,packet)
            }
        }
    }
    return subPackets
}   

packet := new(Packet)
ch := loadData("input_data")

INITIAL_QUEUE_SIZE = len(ch)
packet.Version = int(getBits(ch,3))
packet.Type = int(getBits(ch,3))
VersionSum += packet.Version

switch packet.Type {
case 4: // outer packet is only one literal value
    litVal := getLiteral(ch)
    packet.LiteralValue = litVal
default: // outer packet is an operation
    packet.SubPackets = append(packet.SubPackets, getOperation(ch)...)   
}
PrettyPrint(packet)
fmt.Println(VersionSum)

{
  "Version": 5,
  "Type": 0,
  "LiteralValue": 0,
  "SubPackets": [
    {
      "Version": 6,
      "Type": 4,
      "LiteralValue": 1523,
      "SubPackets": null
    },
    {
      "Version": 3,
      "Type": 1,
      "LiteralValue": 0,
      "SubPackets": [
        {
          "Version": 2,
          "Type": 3,
          "LiteralValue": 0,
          "SubPackets": [
            {
              "Version": 3,
              "Type": 2,
              "LiteralValue": 0,
              "SubPackets": [
                {
                  "Version": 1,
                  "Type": 1,
                  "LiteralValue": 0,
                  "SubPackets": [
                    {
                      "Version": 7,
                      "Type": 0,
                      "LiteralValue": 0,
                      "SubPackets": [
                        {
                          "Version": 0,
                          "Type": 1,
                          "LiteralValue": 0,
                          "Su

              "Type": 4,
              "LiteralValue": 26738628777,
              "SubPackets": null
            }
          ]
        },
        {
          "Version": 7,
          "Type": 4,
          "LiteralValue": 153312376,
          "SubPackets": null
        }
      ]
    },
    {
      "Version": 0,
      "Type": 1,
      "LiteralValue": 0,
      "SubPackets": [
        {
          "Version": 3,
          "Type": 4,
          "LiteralValue": 49096,
          "SubPackets": null
        },
        {
          "Version": 7,
          "Type": 6,
          "LiteralValue": 0,
          "SubPackets": [
            {
              "Version": 0,
              "Type": 4,
              "LiteralValue": 24266,
              "SubPackets": null
            },
            {
              "Version": 1,
              "Type": 4,
              "LiteralValue": 24266,
              "SubPackets": null
            }
          ]
        }
      ]
    },
    {
      "Version": 6,
      "Type": 1,
     

4 <nil>